<a href="https://colab.research.google.com/github/cjc1119/google-classroom/blob/main/Cesar_Cepeda_03_quiz_1015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quiz 3
To **submit** your quiz, submit it in Google Classroom like any other assignment!

## Task:
Your task for quiz 3 is to write python code that:
- Imports the libraries you need
- Reads in the data: data/nba_rookies.csv
- Processes data:
    1. Set the 'Name' column as the index
    2. Converts the 'TARGET_5Yrs' column to 0/1 using 0 for 'No' and 1 for 'Yes'
- Models data using any classification algorithm you would like
    - If you do not know what to do here, feel free to fit a Logistic Regression model with default hyperparameters. But you are free to use any other model or methods that you know if you think that would be better!
    - Use a random state of 42 when splitting your data into training and testing
    - Use ALL columns (except your target) as your X matrix
- Generates predictions on your test data
- Creates a new DataFrame that has:
    1. One column called 'predictions' which is the predictions from your model on your test data
    2. An index that is the name of the player associated with the prediction from your test data
- Writes the DataFrame with the predictions to a csv called 'predictions.csv'

**Comments of the above instructions are included to guide your workflow.**

## Quiz Information
- This is an "open book" quiz - you may use any resources! This includes Jupyter notebooks, Google, StackOverflow, and your notes. ANY RESOURCE YOU USE MUST BE CITED (comment your code with links, which lesson you got the inspiration from, etc.) and failure to cite resources you use will be considered plagiarism.
- You may NOT work with anyone else on this quiz (your classmates, your instructor, etc.).
- You will have **one hour** to take this quiz.
- Do your best and good luck!


In [33]:
# Import the libraries you need
import pandas as pd

# What else?: *sourcing 402-exercise-solution-code

import numpy as np
import matplotlib.pyplot as plt

# 402 Lesson Notebook, LogReg
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split

# 403 Lesson Ntbk, kNN
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [14]:
# Read in the data: data/nba_rookies.csv
rookies_df = pd.read_csv('https://gitlab.com/rowanschaefer/data/-/raw/3f5167221974de09364f214734851923d6c854fe/nba_rookies.csv')
rookies_df.head(), rookies_df.shape

(              Name  GP   MIN  PTS  FGM  FGA   FG%  3P Made  3PA   3P%  ...  \
 0   Brandon Ingram  36  27.4  7.4  2.6  7.6  34.7      0.5  2.1  25.0  ...   
 1  Andrew Harrison  35  26.9  7.2  2.0  6.7  29.6      0.7  2.8  23.5  ...   
 2   JaKarr Sampson  74  15.3  5.2  2.0  4.7  42.2      0.4  1.7  24.4  ...   
 3      Malik Sealy  58  11.6  5.7  2.3  5.5  42.6      0.1  0.5  22.6  ...   
 4      Matt Geiger  48  11.5  4.5  1.6  3.0  52.4      0.0  0.1   0.0  ...   
 
    FTA   FT%  OREB  DREB  REB  AST  STL  BLK  TOV  TARGET_5Yrs  
 0  2.3  69.9   0.7   3.4  4.1  1.9  0.4  0.4  1.3           No  
 1  3.4  76.5   0.5   2.0  2.4  3.7  1.1  0.5  1.6           No  
 2  1.3  67.0   0.5   1.7  2.2  1.0  0.5  0.3  1.0           No  
 3  1.3  68.9   1.0   0.9  1.9  0.8  0.6  0.1  1.0          Yes  
 4  1.9  67.4   1.0   1.5  2.5  0.3  0.3  0.4  0.8          Yes  
 
 [5 rows x 21 columns],
 (1329, 21))

### Process Data

In [15]:
# 1. Set the 'Name' column as the index
rookies_df.set_index('Name', inplace=True)

In [17]:
rookies_df.head(), rookies_df.shape #confirmed 1 col dropped

(                 GP   MIN  PTS  FGM  FGA   FG%  3P Made  3PA   3P%  FTM  FTA  \
 Name                                                                           
 Brandon Ingram   36  27.4  7.4  2.6  7.6  34.7      0.5  2.1  25.0  1.6  2.3   
 Andrew Harrison  35  26.9  7.2  2.0  6.7  29.6      0.7  2.8  23.5  2.6  3.4   
 JaKarr Sampson   74  15.3  5.2  2.0  4.7  42.2      0.4  1.7  24.4  0.9  1.3   
 Malik Sealy      58  11.6  5.7  2.3  5.5  42.6      0.1  0.5  22.6  0.9  1.3   
 Matt Geiger      48  11.5  4.5  1.6  3.0  52.4      0.0  0.1   0.0  1.3  1.9   
 
                   FT%  OREB  DREB  REB  AST  STL  BLK  TOV TARGET_5Yrs  
 Name                                                                    
 Brandon Ingram   69.9   0.7   3.4  4.1  1.9  0.4  0.4  1.3          No  
 Andrew Harrison  76.5   0.5   2.0  2.4  3.7  1.1  0.5  1.6          No  
 JaKarr Sampson   67.0   0.5   1.7  2.2  1.0  0.5  0.3  1.0          No  
 Malik Sealy      68.9   1.0   0.9  1.9  0.8  0.6  0.1  1.0  

In [18]:
# 2. Convert the 'TARGET_5Yrs' column to 0/1 using 0 for 'No' and 1 for 'Yes'
rookies_df['TARGET_5Yrs'].value_counts()

,count
TARGET_5Yrs,
Yes,826
No,503


In [19]:
rookies_df['TARGET_5Yrs'] = rookies_df['TARGET_5Yrs'].map({'No':0, 'Yes':1})

In [20]:
rookies_df['TARGET_5Yrs'].value_counts() # confirmed mapping

,count
TARGET_5Yrs,
1,826
0,503


In [26]:
# quick eda
rookies_df.info() # all dtypes accurate, no nulls

<class 'pandas.core.frame.DataFrame'>
Index: 1329 entries, Brandon Ingram to Jon Barry
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GP           1329 non-null   int64  
 1   MIN          1329 non-null   float64
 2   PTS          1329 non-null   float64
 3   FGM          1329 non-null   float64
 4   FGA          1329 non-null   float64
 5   FG%          1329 non-null   float64
 6   3P Made      1329 non-null   float64
 7   3PA          1329 non-null   float64
 8   3P%          1329 non-null   float64
 9   FTM          1329 non-null   float64
 10  FTA          1329 non-null   float64
 11  FT%          1329 non-null   float64
 12  OREB         1329 non-null   float64
 13  DREB         1329 non-null   float64
 14  REB          1329 non-null   float64
 15  AST          1329 non-null   float64
 16  STL          1329 non-null   float64
 17  BLK          1329 non-null   float64
 18  TOV          1329 non-null   float6

In [28]:
rookies_df.describe() # no outliers, target_5 years mean is 62% yes

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
count,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000
mean,60.402558,17.647931,6.820166,2.634763,5.902483,44.117532,0.249661,0.785628,19.308126,1.302784,1.827314,70.407148,1.007600,2.021445,3.028442,1.558992,0.620391,0.366140,1.196012,0.621520
std,17.481092,8.327441,4.365671,1.686850,3.598820,6.119492,0.384608,1.063856,16.022916,0.989341,1.326655,10.489838,0.779178,1.362464,2.062357,1.474159,0.410533,0.428534,0.724470,0.485191
min,11.000000,3.100000,0.700000,0.300000,0.800000,23.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.300000,0.000000,0.000000,0.000000,0.100000,0.000000
25%,47.000000,10.800000,3.700000,1.400000,3.300000,40.200000,0.000000,0.000000,0.000000,0.600000,0.900000,64.800000,0.400000,1.000000,1.500000,0.600000,0.300000,0.100000,0.700000,0.000000
50%,63.000000,16.100000,5.600000,2.100000,4.800000,43.900000,0.100000,0.300000,22.400000,1.000000,1.500000,71.300000,0.800000,1.700000,2.500000,1.100000,0.500000,0.200000,1.000000,1.000000
75%,77.000000,22.900000,8.800000,3.400000,7.500000,47.800000,0.400000,1.200000,32.500000,1.700000,2.300000,77.600000,1.400000,2.600000,4.000000,2.000000,0.800000,0.500000,1.500000,1.000000
max,82.000000,40.900000,28.200000,10.200000,19.800000,73.700000,2.300000,6.500000,100.000000,7.700000,10.200000,100.000000,5.300000,9.600000,13.900000,10.600000,2.500000,3.900000,4.400000,1.000000


In [30]:
# Use ALL columns (except your target) as your X matrix

# first assessing class imbalance
rookies_df['TARGET_5Yrs'].value_counts(normalize=True) #62% Yes, 37% No; will stratify to adjust for class imbalance

,proportion
TARGET_5Yrs,
1,0.62152
0,0.37848


Baseline LogReg model prediction is therefore target_class = 1, at 62.2%.

In [31]:
# set up X & y
X = rookies_df.drop(columns = 'TARGET_5Yrs')
y = rookies_df['TARGET_5Yrs']

In [32]:
X.shape, y.shape # correct cols / rows for each

((1329, 19), (1329,))

In [35]:
# Use a random state of 42 when splitting your data into training and testing
# in addition to 402 exercise, referring to 402/403 notes
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    random_state=42,
    stratify=y
)

In [36]:
# scaling
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)  # fit-transform only train
X_val_sc = sc.transform(X_val)          # transform only val

### Model data:
Using any classification algorithm you would like.
- If you do not know what to do here, feel free to fit a Logistic Regression model with default hyperparameters, but you are free to use **any other model** or methods that you know if you think that would be better!

From 402: same general linear steps
1. Define X & y
2. Instantiate
3. Fit
4. Generate preds
5. Evaluate

In [23]:
# Fit a classification

In [39]:
# instantiate
# testing default, will adjust if convergence warning
logreg = LogisticRegression()

In [38]:
# instantiating knn as alternate I'd test with more time
knn = KNeighborsClassifier()

In [ ]:
# Generate predictions on your test data


In [40]:
# fit
logreg.fit(X_train_sc, y_train) # using default HPTs (c = 1/alpha = 1; penalty = l2 =Ridge regularization)

LogisticRegression()

In [41]:
# BVT asssessment
# score train
logreg.score(X_train_sc, y_train)

0.7098393574297188

In [42]:
# score test
logreg.score(X_val_sc, y_val)

0.7267267267267268

Train Score -> Test Score
71.0% -> 72.7%
- Slight accuracy improvement on test dataset indicates good BVT balance, not overfit.
  - Could look at slight HPT adjustments to improve further.
- Beats baseline accuracy score of 62.2%.

In [ ]:
# referencing 402 exercise solutions again as noted to investigate coefficients with more time
# exponeentiate to interpret a coeff for a particular column.


### Create a new DataFrame for predictions


In [ ]:
# Have one column called 'predictions'
# which is the predictions from your model on your test data

# referencing 402 exercise solutions again as noted
logreg_preds = logreg.predict(X_val_sc)
logreg_preds

array([1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,

In [45]:
logreg_preds_proba = logreg.predict_proba(X_val_sc)
logreg_preds_proba

array([[0.44094673, 0.55905327],
       [0.67737488, 0.32262512],
       [0.14695631, 0.85304369],
       [0.50319267, 0.49680733],
       [0.41439749, 0.58560251],
       [0.33393243, 0.66606757],
       [0.79568553, 0.20431447],
       [0.40145625, 0.59854375],
       [0.51512564, 0.48487436],
       [0.54877826, 0.45122174],
       [0.60487472, 0.39512528],
       [0.14090307, 0.85909693],
       [0.85082959, 0.14917041],
       [0.29710211, 0.70289789],
       [0.57259029, 0.42740971],
       [0.53042748, 0.46957252],
       [0.20412246, 0.79587754],
       [0.59383937, 0.40616063],
       [0.4675801 , 0.5324199 ],
       [0.02128196, 0.97871804],
       [0.54673613, 0.45326387],
       [0.29871434, 0.70128566],
       [0.65379118, 0.34620882],
       [0.56360343, 0.43639657],
       [0.18456132, 0.81543868],
       [0.73037759, 0.26962241],
       [0.7240508 , 0.2759492 ],
       [0.09394043, 0.90605957],
       [0.27012701, 0.72987299],
       [0.53850568, 0.46149432],
       [0.

In [49]:
# Have an index that is the name of the player
predictions_df = pd.DataFrame(logreg_preds, index=X_val.index)
predictions_df.head()

,0
Name,
Jerrod Mustaf,1
Everette Stephens,0
Isiah Thomas*,1
Sean Higgins,0
Kris Humphries,1


In [52]:
predictions_df.rename(columns = {0:'predictions'}, inplace=True)
predictions_df.head() # assisted by Colab


,predictions
Name,
Jerrod Mustaf,1
Everette Stephens,0
Isiah Thomas*,1
Sean Higgins,0
Kris Humphries,1


In [53]:
# Write the DataFrame you created to a csv called 'predictions.csv'
predictions_df.to_csv('predictions.csv') # assisted by Colab
